In [2]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

In [3]:
summit = analyze.Analysis('results/summit')
print(summit)
some_results = summit.load_results(('testem3-flat','orange','gpu'), 0)

Analysis for Celeritas v0.1.3+609057e6 on summit


In [6]:
mp_per_gpu = some_results['system']['device']['multiprocessor_count']

In [ ]:
inp = some_results['input']
primaries_per_event = inp['primary_gen_options']['primaries_per_event']
def calc_event_rate(summary):
    event_rate = analyze.inverse_summary(summary['avg_time_per_primary'])
    event_rate['mean'] /= primaries_per_event
    event_rate['std'] /= primaries_per_event
    return event_rate

In [ ]:
summit.result.columns

In [ ]:
for key, lines in summit.failures()['stderr'].iteritems():
    print("="*78)
    print(key)
    print("   " + " - \n".join(lines[-3:]))

In [ ]:
summit.failures()['stderr'].groupby(['problem', 'geo', 'arch']).count().unstack()

In [ ]:
summed = analyze.summarize_instances(summit.result[[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time',
    'unconverged',
    'slot_occupancy']])
#summed.xs('mean', axis=1, level=1).to_csv('summit.csv')

In [ ]:
a = {}
a.setdefault('foo', 0)

In [ ]:
summit.result['num_primaries'].xs(('gpu', 0), level=('arch', 'instance'))

In [ ]:
summed[('total_time', 'mean')].unstack()

In [ ]:
summed['avg_time_per_primary'].xs('testem3-flat', level='problem')

In [ ]:
#rel_err = summed.xs('std', axis=1, level=1) / summed.xs('mean', axis=1, level=1)
#high_err = rel_err > 0.05
#rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

In [ ]:
unconv = summed['unconverged']
unconv[unconv['mean'] > 0]

In [ ]:
summit.result.xs('cms2018+field+msc', level='problem')[['avg_steps_per_primary', 'slot_occupancy']]

In [ ]:
problems = summit.problems()
p_to_i = dict(zip(problems, itertools.count()))
g_to_color = {'orange': '#F6A75E', 'vecgeom': '#5785B7'}
arch_to_shape = {'gpu': 'x', 'cpu': 'o'}

In [ ]:
problem_to_abbr = {'testem15': 'A',
 'testem15+field': 'AF',
 'testem15+field+msc': 'AFM',
 'simple-cms+msc': 'B',
 'simple-cms+field': 'BF',
 'simple-cms+field+msc': 'BFM',
 'testem3-flat': 'C',
 'testem3-flat+field': 'CF',
 'testem3-flat+msc': 'CM',
 'cms2018': 'Z',
 'cms2018+field+msc': 'ZFM'}
# TODO: revert when rerunning to get updated 
# problem_abbr = [problem_to_abbr[k] for k in problems]
problems = list(problem_to_abbr.keys())
p_to_i = dict(zip(problems, itertools.count()))
problem_abbr = list(problem_to_abbr.values())
print('\n'.join('\t'.join([v, k]) for (k, v) in problem_to_abbr.items()))

In [ ]:
def plot_results(ax, summary):
    index = np.array([p_to_i[p]
                      for p in summary.index.get_level_values('problem')], dtype=float)
    index += [(0.1 if g == 'orange' else -0.05)
              for g in summary.index.get_level_values('geo')]
    color = np.array([g_to_color[g]
                      for g in summary.index.get_level_values('geo')])
    
    if 'arch' in summary.index.names:
        slc_mark = [(a.upper(), summary.index.get_level_values('arch') == a, arch_to_shape[a])
                    for a in ['cpu', 'gpu']]
        
    else:
        slc_mark = [(None, slice(None), 's')]

    result = []
    for lab, slc, mark in slc_mark:
        temp_idx = index[slc]
        temp_sum = summary.loc[slc]
        ax.errorbar(temp_idx, temp_sum['mean'], temp_sum['std'],
                    capsize=0, fmt='none', ecolor=(0.2,)*3)
        scat = ax.scatter(temp_idx, temp_sum['mean'], c=color[slc], marker=mark,
                         label=lab)
        result.append(scat)
    
    xax = ax.get_xaxis()
    xax.set_ticks(np.arange(len(problems)))
    xax.set_ticklabels(problem_abbr, rotation=90)
    grid = ax.grid()
    ax.set_axisbelow(True)
    return scat

In [ ]:
speedup = analyze.get_cpugpu_ratio(summed['total_time'])
fig, ax = plt.subplots()
plot_results(ax, speedup)
ax.set_ylabel("Speedup (7-CPU / 1-GPU wall time)")
analyze.annotate_metadata(ax, summit);
fig.savefig('plots/speedups.pdf', transparent=True)
plt.close()

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(4,4), subplot_kw=dict(yscale='log'))
for (ax, q) in zip(axes, ['step', 'primary']):
    plot_results(ax, analyze.inverse_summary(summed['avg_time_per_' + q]))
    ax.set_ylabel(q + ' per sec')
    ax.legend()
fig.savefig('plots/steps-vs-primaries.png', dpi=300)
plt.close()

In [ ]:
event_rate = calc_event_rate(summed)

In [ ]:
event_rate.xs('testem3-flat', level='problem')

In [ ]:
(fig, (time_ax, occ_ax)) = plt.subplots(
    nrows=2, figsize=(4, 4),
    gridspec_kw=dict(height_ratios=[3, 1])
)
time_ax.set_yscale('log')
plot_results(time_ax, event_rate)
time_ax.set_ylabel(r"Event rate [1/s]")
time_ax.legend()
time_ax.set_xticklabels([])
plot_results(occ_ax, summed['slot_occupancy'])
occ_ax.set_ylabel("Slot occupancy")
analyze.annotate_metadata(ax, summit)
fig.savefig('plots/rate-occupancy.pdf', transparent=True)
plt.close()

In [ ]:
speedup.dropna().applymap("{:.1f}".format)

## Action fraction pie charts


In [ ]:
mean_action_times = summit.action_times().xs('mean', axis=1, level=1).T
mean_action_times.sort_index(inplace=True)

In [ ]:
for ext in ["", "+field+msc"]:
    prob_geo = ('cms2018'+ext, 'vecgeom')
    temp = mean_action_times.xs(prob_geo, axis=1, level=('problem', 'geo')).dropna()

    for (arch, series) in temp.iteritems():
        (fig, ax) = plt.subplots(figsize=(4, 4))
        ax.pie(series, labels=series.index, autopct='%1.1f%%', pctdistance=0.85)
        ax.axis('equal')
        name = prob_geo + (arch,)
        slashname = "/".join(name)
        fig.text(
            0.98, 0.02, f"{slashname}\n{summit.version} on {summit.system}",
            va='bottom', ha='right',
            fontstyle='italic', color=(0.5,)*3, size='xx-small',
            zorder=-100
        )
        dashname = "-".join(name)
        fig.savefig(f'plots/{dashname}.pdf', transparent=True)
        plt.close()

### Plot per-step timing on GPU

In [ ]:
cms = [summit.load_results((p, 'vecgeom', 'gpu'), 0)
       for p in ['cms2018', 'cms2018+field+msc']]

for plot, label in [(analyze.plot_counts, 'counts'),
                    (analyze.plot_accum_time, 'time')]:
    (fig, axes) = plt.subplots(ncols=2, figsize=(8, 2))
    
    for (i, ax, data) in zip(itertools.count(), axes, cms):
        objs = plot(ax, data)
        analyze.annotate_metadata(ax, data['_metadata'])
        if i == 0:
            objs['oax'].set_ylabel(None)
        elif i == 1:
            objs['ax'].set_ylabel(None)
    fig.savefig(f'plots/cms-{label}.pdf', transparent=True)
    plt.close()

## Crusher

In [ ]:
crusher = analyze.Analysis('results/crusher')
print(crusher)

In [ ]:
crusher_fail = crusher.failures().xs('orange', level='geo')

In [ ]:
for key, lines in crusher_fail['stderr'].iteritems():
    print("="*78)
    print(key)
    print("   " + " - \n".join(lines[-3:]))

In [ ]:
crusher_fail['stderr'].groupby(['problem', 'arch']).count().unstack()

In [ ]:
csum = analyze.summarize_instances(crusher.result[~crusher.invalid][[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time',
    'unconverged',
    'slot_occupancy',
]])

In [ ]:
csum[('total_time', 'mean')].unstack()

In [ ]:
rel_err = csum.xs('std', axis=1, level=1) / csum.xs('mean', axis=1, level=1)
high_err = rel_err > 0.02
rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

In [ ]:
analyze.get_cpugpu_ratio(csum['total_time'])

In [ ]:
crusher_times = csum['total_time']
crusher_times

In [ ]:
crusher_rates = calc_event_rate(csum)
summit_rates = calc_event_rate(summed.loc[crusher_times.index])

counts = {
    ('summit', 'cpu'): 7,
    ('summit', 'gpu'): 1,
    ('crusher', 'cpu'): 8,
    ('crusher', 'gpu'): 1,
}

In [ ]:
(crusher_rates['mean'] / summit_rates['mean']).unstack()

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
for offset, color, machine, rates in [(-0.05, '#7A954F', 'Summit', summit_rates),
                                      (0.05, '#BC5544', 'Crusher', crusher_rates)]:
    for arch in ['cpu', 'gpu']:
        summary = rates.xs(arch, level='arch')
        index = np.array([p_to_i[p]
                          for p in summary.index.get_level_values('problem')], dtype=float)
        index += offset
    
        mark = arch_to_shape[arch]
        count = counts[(machine.lower(), arch)]
        arch = arch.upper()
        ax.errorbar(index, summary['mean'], summary['std'],
                    capsize=0, fmt='none', ecolor=(0.2,)*3)
        scat = ax.scatter(index, summary['mean'], c=color, marker=mark,
                         label=f"{machine} ({count} {arch})")    
xax = ax.get_xaxis()
xax.set_ticks(np.arange(len(problems)))
xax.set_ticklabels(problem_abbr, rotation=90)
grid = ax.grid()
ax.set_axisbelow(True)
ax.legend()
ax.set_ylabel(r"Event rate [1/s]")
analyze.annotate_metadata(ax, summit)
fig.savefig('plots/crusher-vs-summit.pdf')
plt.close()